In [1]:
# Import the required libraries
import pandas as pd
import numpy as np

In [2]:
# From a Gnomad file , open and create a dataframe with all the information
def file_to_df(file):
    file = open(file,'r')
    lines = file.readlines()
    file.close()
    df = pd.read_csv('Match(Gnomad)_nonPatho_Pathogenic.txt', delimiter = " ")
    df.rename(columns={'HGVS Consequence': 'HGVSConsequence'}, inplace=True)
    df = df[df['HGVSConsequence'].notna()] # Takes all rows from HGVS_Consequence which are not NA
    return(df)

In [3]:
# This function firts selects the columns that are from our interest in the df --> "Gene(s)", "HGVS_Consequence"
# Then separates the column Port_change which looks like this: R630* into -->
#       inital_aa: R, position: 630 and final_aa: blank(It does not identify the *)
# Then it replaces blank spaces with NAs to after that replace the NAs with TER
# To finish, it just gets the columns which we're interested in
def df_to_dfwanted(df):
    gene_protchange = df[["Gene(s)", "HGVSConsequence"]]
    gene_protchange[['initial_aa', 'position', 'final_aa']] = gene_protchange['HGVSConsequence'].str.extract('([A-Za-z]+)(\d+\.?\d*)([A-Za-z]*)', expand = True)
    gene_protchange = gene_protchange.replace('', np.nan)
    gene_protchange['final_aa'].fillna('TER', inplace=True)
    gene_protchange = gene_protchange[["Gene(s)", "position", "initial_aa", "final_aa"]]
    return(gene_protchange)

In [4]:
def dfwanted_to_output(gene_protchange):
    gene_protchange.to_csv('Match(Gnomad)_nonPatho_Pathogenic_final.txt', header=True, index=None, sep=' ', mode='w')

In [5]:
# The required items are obtained from a file downloaded from Gnomad are given as an output in .txt
df = file_to_df('Match(Gnomad)_nonPatho_Pathogenic.txt')
gene_protchange = df_to_dfwanted(df)
gene_protchange['position'] = gene_protchange['position'].astype(str)
out = dfwanted_to_output(gene_protchange)